# 1. load/reload all dependency

In [1]:
import numpy as np
import pandas as pd
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import sys
sys.path.append("../")
import pyhealth.datasets.datasets as datasets
import pyhealth.models.models as models
from importlib import reload
reload(datasets)
reload(models)
import torch
print(torch.cuda.is_available())


True


# 2. load dataset and initialize model

In [2]:
# load MIMIC-III
dataset = datasets.MIMIC_III()

# initialize the model and build the dataloaders
dataset.get_dataloader("GAMENet")
model = models.GAMENet(
    dataset=dataset,
    emb_dim=64,
)

loaded the med table!
loaded the diag table!
loaded the prod table!
generated .pat_to_visit!
generated .visit_dict!
----- preparing code mappings -----
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 2.4580435752868652s
-----------------------------------------
generated .encoded_visit_dict!
generated .maps (for code to index mappings)!
load severe ddi pairs from https://drive.google.com/uc?id=1R88OIhn-DbOYmtmVYICmjBSOIsEljJMh!
ddi info is from https://drive.google.com/file/d/1mnPc0O0ztz0fkv3HF-dpmBb8PLWsEoDz/view?usp=sharing!
generated train/val/test dataloaders for GAMENet model!


# 3. train and test

In [3]:
# set trainer with checkpoint
checkpoint_callback = ModelCheckpoint(dirpath='./model_cpt')
trainer = Trainer(
    gpus=1,
    max_epochs=3,
    progress_bar_refresh_rate=5,
    callbacks=[checkpoint_callback],
)

# train model
trainer.fit(
    model=model,
    train_dataloaders=dataset.train_loader,
    val_dataloaders=dataset.val_loader,
)

# test the best model
model.summary(
    test_dataloaders=dataset.test_loader,
    ckpt_path=checkpoint_callback.best_model_path,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name       | Type       | Params
------------------------------------------
0 | embeddings | ModuleList | 551 K 
1 | dropout    | Dropout    | 0     
2 | encoders   | ModuleList | 49.9 K
3 | query      | Sequential | 8.3 K 
4 | ehr_gcn    | _GCN       | 71.4 K
5 | ddi_gcn    | _GCN       | 71.4 K
6 | output     | Sequential | 46.4 K
------------------------------------------
685 K     Trainable params
141 K     Non-trainable params
826 K     Total params
3.308     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

--- Test Summary ---
DDI rate: 0.08581
Jaccard: 0.4236
PRAUC: 0.7306
AVG_PRC: 0.6929
AVG_RECALL: 0.55
AVG_F1: 0.5902
AVG_MED: 27.75

